Build a model step by step by your custom dataset
* get data (download)
* (optional) using PIL or matplotlib can visualize img
* make train and test directory
* create dataset (using torchvsion.datasets) from train and test directory
* create Dataloader (using utils.data.DataLoader) from dataset
* (optional) using matplotlib check image shape and others.
*build model (using torch.nn)
* send data to device(cuda- gpu) if available and instantiate an instance (object) for above move
* test the model with one image to check model is working fine or not
*crate train_step(), test_step() and train () function for training model
* create a function to save model
* train and evaluate and save the model
* visualize model perfomance

In [ ]:

# 1. get data
import os
import zipfile
from pathlib import Path
import requests
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
if image_path.is_dir():
  print(f"{image_path} directory exists.")
else:
  print(f"Did not find {image_path} directory, creating one...")
  image_path.mkdir(parents=True, exist_ok = True)
with open(data_path / "pizza_steak_sushi.zip" , "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("downloading data")
  f.write(request.content)
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("unzipping file")
  zip_ref.extractall(image_path)
os.remove(data_path / "pizza_steak_sushi.zip")

In [10]:
!python get_data.py

data/pizza_steak_sushi directory exists.
downloading data
unzipping file


In [ ]:
#optional visualilze download image

In [ ]:
#2. make train and test directory
train_dir = image_path /"train"
test_dir = image_path /"test"

In [ ]:
#3.create dataset with transform 
from torchvision import datasets, transforms
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

train_data = datasets. ImageFolder(root = train_dir,
                                   transform = data_transform,
                                   target_transform = None)
test_data = datasets.ImageFolder(root = test_dir,
                                 transform = data_transform)
train_data, test_data
class_names = train_data.classes
class_names
class_dict = train_data.class_to_idx
class_dict
len(train_data), len(test_data)


In [ ]:
# 4.turn dataset into DataLoader
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset = train_data,
                              batch_size =32,
                              num_workers = 1
                              shuffle = True)
test_dataloader = DataLoader(dataset = test_data,
                             batch_size = 32,
                             num_workers =1,
                             shuffle = False)
train_dataloader, test_dataloader
#check single image in dataloader
img, label = next(iter(train_dataloader))
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

In [ ]:
#5 build a model
import torch
from torch import nn
class TinyVGG(nn.Module):
  def __init__(self, input_shape:int, hidden_units: int, output_shape:int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                  out_channels = hidden_units,
                  kernel_size = 3,
                  stride =1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size =3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units *13*13,
                  out_features = output_shape)
    )
    def forward(self, x: torch.Tensor):
      #x = self.conv_block_1(x)
      #x = self.conv_block_2(x)
      #x = self.classifier(x)
      return self.classifier(self.conv_block_2(self.conv_block_1(x)))

In [ ]:
# 6. cuda availabe and instantiate an instance of the model
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
model_0 = TinyVGG(input_shape =3,
                  hidden_units =10,
                  output_shape = len(train_data.classes)).to(device)
model_0

In [ ]:
# 7. test model by passing one image through our above created model
#one image you will get from dataloader
img_batch, label_batch = next(iter(train_dataloader))
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"single image shape: {img_single.shape}\n")
model_0.eval()
with torch.inference_mode():
  pred = model_0(img_single.to(device))
print(f"output logits:\n {pred}\n")
print(f"output prediction probabilites:\n {torch.softmax(pred, dim=1)}\n")
print(f"output prediction label:\n {torch.argmax(torch.softmax(pred, dim=1),dim=1)}\n")
print(f"Actual label: \n {label_single}")


In [ ]:
#8. train_step function
from typing import Tuple
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
  model.train()
  train_loss,train_acc = 0, 0
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1),dim=1)
    train_acc += (y_pred_class ==y).sum().item() /len(y_pred)
  train_loss = train_loss /len(dataloader)
  train_acc = train_acc /len(dataloader)
  return train_loss, train_acc


In [ ]:
#test loop
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):
  model.eval()
  test_loss, test_acc = 0,0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      test_loss += loss.item()
      y_pred_class = torch.argmax(torch.softmac(y_pred, dim=1), dim=1)
      test_acc += (y_pred_class ==y).sum().item() / len(y_pred)
  test_loss = test_loss /len(dataloader)
  test_acc = test_acc /len(dataloader)
  return test_loss, test_acc

In [ ]:
#now combine train_step and test_step functin inside a function
from typing import Dict, List
from tqdm.auto import tqdm
def train(model: torch.nn.Module,
           train_dataloader: torch.utils.data.DataLoader,
           test_dataloader: torch.utils.data.DataLoader,
           optimizer: torch.optim.Optimizer,
           loss_fn: torch.nn.Module,
           epochs: int,
           device: torch.device):
   results = {"train_loss" :[],
              "train_acc" : [],
              "test_loss" :[],
              "test_acc" : []
              }
    #loop above function
   for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                         dataloader = train_dataloader,
                                         loss_fn = loss_fn,
                                         optimizer = optimizer,
                                         device = device)
      test_loss, test_acc = test_step(model = model,
                                      dataloader = test_dataloader,
                                      loss_fn = loss_fn,
                                      device = device)
      print(f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f}  | "
            f"train_acc:  {train_acc: .4f}  | "
            f"test_loss:  {test_loss: .4f}  |"
            f"test_acc:   {test_acc: .4f}" )
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)
   return results
  

In [ ]:
#9. save the model
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents = True,
                        exist_ok = True)
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path /model_name
  print(f"[Info] saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)  

In [ ]:
#10. train, evaluate and save the model
torch.manual_seed(42)
torch.cusa.manual_seed(42)
NUM_EPOCHS = 5
model_0 = TinyVGG(input_shape=3,
                  hidden_units =10,
                  output_shape = len(train_data.classes)).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr = 0.001)
from timeit import default_timer as timer
start_time = timer()
model_0_results = train(model=model_0,
                        train_dataloader = train_dataloader,
                        test_dataloader = test_dataloader,
                        optimizer = optimizer,
                        loss_fn = loss_fn,
                        epochs = NUM_EPOCHS,
                        device = device)
end_time = timer()
print(f"[info] total training time : {end_time - start_time:.3f} seconds")
save_model(model=model_0,
           target_dir = "models",
           model_name = "05_going_modular_cell_mode_tinyvgg_model.pth")



In [ ]:
#11 visualization test loss curve and accuracy curve